In [1]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [2]:
class NullModel(Module):
    def __init__(self):
        pass
    def forward(*x):
        return x

class NullLoss(Module):
    def __init__(self):
        pass
    def forward(*x):
        y = tensor(0.)
        y.requires_grad = True
        return y
    
def null_splitter(m):
    return [[],[]]

In [3]:
def read_data(eugenio):
    path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv' if eugenio else '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()
    return df 

def get_vid_path(df:pd.Series): return df['paths']
def get_lbls(df:pd.Series): return df['lbl']

path_data = Path('/mnt/data/adrianlopez/Videos/Charades/df.csv')
df = read_data(False)

In [4]:
tfms = [[get_vid_path, Video.create],
        [get_lbls, Categorize()]]
splits = RandomSplitter(seed=42)(df)
dsets = Datasets(df, tfms, splits=splits)

In [5]:
dsets[0][0]

(#221) [PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180,PILImage mode=RGB size=320x180...]

In [6]:
dls = dsets.dataloaders(n_lbl=4, n_el=4, 
                        dl_type=UniformizedDataLoader,
                        after_item=[ResizeTime(l=20), Resize(224), ToTensor()],
                        after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

In [7]:
b = dls.one_batch()
print(b[0].shape)
print(b[1])

torch.Size([16, 3, 20, 224, 224])
TensorCategory([ 21,  21,  21,  21, 110, 110, 110, 110,  54,  54,  54,  54,  33,  33,
         33,  33], device='cuda:0')


In [7]:
head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
learn = Learner(dls, 
#                 NullModel(),
                TLModel(body,head),
#                 loss_func=NullLoss(),
                MixedLoss(alpha=1.0, margin=.2),
#                 splitter=null_splitter,
                splitter=my_splitter,
#                 opt=Optimizer([0], noop),
#                 metrics=lambda *x: 0)
                metrics=tl_accuracy)

In [ ]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,tl_accuracy,time


In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_datasets.ipynb.
Converted 05_first_inflated_NN.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp first_inflated_NN